In [ ]:
#!conda activate py38_estnltk1.7

# TODO

* küsisõnade count (küsin Kadril ja Heikilt)
* rääkida Heikilt, mida ta tabelisse veel tahaks
* clauses_count - siis kui tuleb uus kiht


## Tabeli struktuur
|  |  |  |
|:---|:---|:---|
|col_id| int | kollektsiooni ID PostgreSQL andmebaasis |
|root_id | int |OBL juure ID |
|word_count| int | kõigi sõnede arv OBL fraasis |
|consistent| int \[0,1\] | OBL fraasi sõned on: <br> 1 -  lauses järjest<br>0 - pole järjest|
|punctuation_count| int | <code>deprel=punct</code> sõnede arv OBL fraasis |
|consistent_wo_punctuation| int \[0,1\] | kui OBL fraasist eemaldada kõik <code>deprel=punct</code> tipud, siis järele jäänud sõned on:<br> 1 - lauses järjest<br>0 - pole järjest
|verb_normal_count| int | *"normaalsete"* tegusõnade arv OBL fraasis |
|verb_unnormal_count| int | *"ebanormaalsete"* tegusõnade arv OBL fraasis|

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from estnltk.storage.postgres import PostgresStorage, IndexQuery, LayerQuery
from data_helpers.db_reader import DbReader
from data_helpers.utils import ListUtils
from data_helpers.syntax_graph import SyntaxGraph
import pandas as pd
from datetime import datetime

collection_name = 'koondkorpus_sentences' 
collection_name = 'koondkorpus_sentences_test_5000_sg_thread'  # 

In [3]:
my_db_reader = DbReader(pgpass_file='~/.pgpass',\
                          schema='estonian_text_corpora',\
                          role='estonian_text_corpora_read',\
                          temporary=False,\
                          collection_name=collection_name)
my_db_reader.set_layers(['v171_named_entities', 'v172_stanza_syntax', 'v172_obl_phrases', 'v172_pre_timexes'])

INFO:storage.py:41: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:58: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


In [ ]:
#collection_size

In [10]:
date_time = datetime.now().strftime("%Y%m%d-%H%M%S")

data = []
columns = [
    'col_id', 
    'root_id', 
    'word_count', 
    'consistent', 
    'punctuation_count', 
    'consistent_wo_punctuation',
    'verb_normal_count',
    'verb_unnormal_count',
]

#for colId, text in my_db_reader.get_collections(count=10000, shuffle=True, queries=[LayerQuery('v172_stanza_syntax', deprel='obl')]):
for colId, text in my_db_reader.get_collections(shuffle=False, queries=[LayerQuery('v172_stanza_syntax', deprel='obl')], progressbar='ascii'):    
    graph = SyntaxGraph(text['v172_stanza_syntax'])
    #vb hoopis kõik deprel count
    punctuation_nodes = graph.get_nodes_by_attributes(attrname='deprel', attrvalue='punct')
    verb_nodes = graph.get_nodes_by_attributes(attrname='pos', attrvalue='V')
    
    for obl in graph.get_obl_info(text['v172_obl_phrases']): 
        is_consistent = ListUtils.is_list_consecutive(obl['nodes'])
        # has_nested_obl = 
        obl_verbs = ListUtils.list_intersection(verb_nodes, obl['nodes'])
        verb_normal_count = len([graph.is_verb_normal(v) for v in obl_verbs])
    
        obl_punctuation = ListUtils.list_intersection(punctuation_nodes, obl['nodes'])
        contains_verb = len(ListUtils.list_intersection(verb_nodes, obl['nodes']))>0
        if not len(obl_punctuation):
            is_consistent_wo_punct = is_consistent
        else:
            obl_nodes_punct_removed = [n for n in obl['nodes'] if n not in obl_punctuation ]
            is_consistent_wo_punct = ListUtils.is_list_consecutive(obl_nodes_punct_removed)
        
        
        r = {
            'col_id': colId, 
            'root_id': obl['root_id'], 
            'word_count': len(obl['nodes']), 
            'consistent': int(is_consistent), 
            'punctuation_count': len(obl_punctuation), 
            'consistent_wo_punctuation': int(is_consistent_wo_punct),
            'verb_normal_count': verb_normal_count,
            'verb_unnormal_count': len(obl_verbs) - verb_normal_count,
        }
            
        data.append(tuple([r[c] for c in columns]))



collection_id: 159735: 100%|########################################################################################################################################| 91550/91550 [07:32<00:00, 202.26doc/s]


In [11]:
df = pd.DataFrame(data=data,  columns=columns)
print(filename)
df.to_csv(filename, index=None)
df.sample(50)

obl_compund_data_koondkorpus_sentences_test_5000_sg_thread_20230519-095846.csv


,col_id,root_id,word_count,consistent,punctuation_count,consistent_wo_punctuation,verb_normal_count,verb_unnormal_count
6871,5033,1,1,1,0,1,0,0
121924,86266,18,2,1,0,1,0,0
176722,135084,6,2,1,0,1,0,0
151155,110130,4,1,1,0,1,0,0
121550,85972,17,5,1,0,1,0,0
152996,111758,11,3,1,0,1,0,0
81533,55339,8,2,1,0,1,0,0
171927,129807,2,3,1,0,1,0,0
36810,25011,3,2,1,0,1,0,0
72488,48997,13,4,1,0,1,0,0
